In [ ]:
#PIA_ENTRENAMIENTO DE UNA RED NEURONAL CONVOLSIONAL      EQUIPO 4      N1
#Carlos Emilio Orozco Ibarra	          1905750
#Jonathan Raziel  Treviño Cruz	        1949173
#Joselyn Anahí Ascencio Valenciano    	2035453
#Josue Miguel Pérez	                    1966557
#Yaros Miroslava Escobedo Hernández   	1955398
#EJERCICIO Entrenar una red neuronal convolucional en un cmjunto de datos de imagnes de frutas en este caso.

In [ ]:
!unzip archive.zip


unzip:  cannot find or open archive.zip, archive.zip.zip or archive.zip.ZIP.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import warnings
warnings.filterwarnings("ignore")
import os
from matplotlib.image import imread
import random
import matplotlib.image as mpimg

In [ ]:
tf.random.set_seed(5)

In [ ]:
datagen = ImageDataGenerator(rotation_range=10,
            rescale = 1./255,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True,
            vertical_flip=False,
            zoom_range=0.1,
            shear_range=0.1,
            brightness_range=[0.8, 1.2],
            fill_mode='nearest',
            validation_split=0.2  # set validation split to 20% 
            )

In [ ]:
trainimagedata = datagen.flow_from_directory("/content/images",
                                              batch_size = 32,
                                              class_mode = 'categorical',
                                              target_size=(64,64),
                                              subset = 'training'
                                            )

testimagedata = datagen.flow_from_directory("/content/images",
                                              batch_size = 32,
                                              class_mode = 'categorical',
                                              target_size=(64,64),
                                              subset = 'validation'
                                            )

In [ ]:
trainimagedata.classes

In [ ]:
trainimagedata.class_indices

In [ ]:
# Print Each sample of all Classes
# Directory path where the images are located
directory_path = '/content/images'

# List of class names
class_names = ['apple fruit', 'banana fruit', 'cherry fruit', 'chickoo fruit', 'grapes fruit', 'kiwi fruit', 'mango fruit', 'orange fruit', 'strawberry fruit']

# Create a figure to display the images
fig = plt.figure(figsize=(12, 8))

# Iterate over each class
for i, class_name in enumerate(class_names):
    # Get a list of image files in the class directory
    class_directory = os.path.join(directory_path, class_name)
    image_files = os.listdir(class_directory)

    # Select a random image file from the class
    random_image = random.choice(image_files)
    image_path = os.path.join(class_directory, random_image)

    # Load and display the image
    ax = fig.add_subplot(3, 3, i+1)
    img = mpimg.imread(image_path)
    ax.imshow(img)
    ax.set_title(class_name)
    ax.axis('off')

# Adjust the layout and display the figure
plt.tight_layout()
plt.show()

In [ ]:
input_shape = trainimagedata.image_shape

In [ ]:
# Model Architecture
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(64,(3,3), input_shape = input_shape,activation='relu',padding='same'))
model.add(tf.keras.layers.MaxPool2D(2,2))

model.add(tf.keras.layers.Conv2D(32,(3,3),activation='relu'))
model.add(tf.keras.layers.MaxPool2D(2,2))

model.add(tf.keras.layers.Flatten())

model.add(Dense(128,activation = 'relu'))
model.add(Dropout(0.25))

model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.25))

model.add(Dense(9,activation = 'softmax'))

In [ ]:
# Set the Hyperparameter to Adam optimizer
from tensorflow.keras.optimizers import Adam
optimizer = Adam(lr=0.001,beta_1=0.9,beta_2 = 0.999, epsilon=1e-8)

In [ ]:
# Compile the model
model.compile(optimizer = optimizer,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor = 'val_loss', patience =15)

In [ ]:
# Fitting the model
mdl_history = model.fit(trainimagedata,
                          validation_data = testimagedata,
                          epochs=35,
                          batch_size=16,
                          callbacks=[early_stop])

In [ ]:
model.summary()

In [ ]:
# Let's visualize training and testing accuracy by graph
plt.plot(mdl_history.history['accuracy'])
plt.plot(mdl_history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Model prediction
y_preds = model.predict(testimagedata)

In [ ]:
# Model Evaluation With metrcics
y_eva = model.evaluate(testimagedata)

In [ ]:
# Let's check our model predict kiwi fruit Class or not (Actual kiwi fruit class is 5)
from tensorflow.keras.preprocessing import image
new_image = image.load_img("/content/images/kiwi fruit/Image_11.jpg",target_size = (64,64))
image_array = image.img_to_array(new_image)
image_final = np.expand_dims(image_array ,axis=0)
image_final /= 255.

In [ ]:
img_pred = model.predict(image_final)

In [ ]:
sample_pred = np.argmax(img_pred)

In [ ]:
sample_pred